To run plotly/widgets in jupyterlab:
``` shell
pip install jupyterlab "ipywidgets>=7.5"
curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -
sudo apt-get install -y nodejs
jupyter labextension install jupyterlab-plotly@4.14.3
```

# Unavailable Space testing Testing

### Imports

In [20]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from willitfit.plotting import plotter, plot_helper

from willitfit.params import VOL_UNAVAILABLE, VOL_EMPTY

In [10]:
data = pd.read_csv('cars_clean.csv')

## Helper Functions

In [11]:
def generate_wireframe(coords_arr):
    '''Quickly generate wireframe from coords arr'''
    meshes = []
    line_marker = dict(color='#0066FF', width=2)
    for arr in coords_arr:
        x,y,z = arr
        meshes.append(go.Scatter3d(x=x,
                                y=z,
                                z=y,
                                mode='lines',
                                #color='grey',
                                line=line_marker))

    return meshes

In [23]:
def test_unavailable_space(car_model, data=data, ratio_door=0.2):
    test_vol = get_volume_space(car_model=car_model, data=data, ratio_door=ratio_door)
    unavail_coords = np.argwhere(test_vol == VOL_UNAVAILABLE).T
    wireframe = generate_wireframe([unavail_coords])

    return plotter.draw_3d_plot(wireframe, test_vol.shape)

# Test

In [22]:
def get_volume_space(car_model, data, ratio_door=0.2):
    """
    Returns available volume for a specific car_model.
    Use ratio_door to tweak estimated unavailable space caused by 45-degree trunk-door slope.
    """
    # Isolate dimension cols
    dim_cols = ['depth', 'height', 'width']

    # Trunk dimensions
    model_row = data[data['car_model'] == car_model]
    trunk_dims = model_row[dim_cols].to_numpy(int)[0]

    # Cuboid Volume Space
    volume_space = np.full(trunk_dims, VOL_EMPTY, dtype=int)

    # Unavailable space
    depth_to_block = int(trunk_dims[0]*ratio_door)
    for i in range(-depth_to_block, 0, 1):
        volume_space[i,-i:,:] = VOL_UNAVAILABLE

    return volume_space

In [17]:
data.iloc[4]['car_model']

'218D GRAN TOURER LUXURY LINE STEPTRONIC 5DR MPV 2015'

In [24]:
test_unavailable_space(data.iloc[4]['car_model'], data, 0.3)

TypeError: draw_3d_plot() takes 1 positional argument but 2 were given

In [ ]:
fom